In [ ]:
QUERY = """
    SELECT * FROM scores;
"""
df = print_table(QUERY)

In [ ]:
# Overall Score = 0.2 * sentiment_score + 0.25 * trend_score + 0.1 * location_score + 0.35 * rating_score + 0.1 * price_score

df["overall_score"] = (0.25 * df["sentiment_score"]) + (0.05 * df["trend_score"]) + (0.1 * df["location_score"]) + (0.4 * df["rating_score"]) + (0.05 * df["price_score"]) + (0.15 * df["uniqueness_score"])

In [ ]:
# Update score table

conn = sqlite3.connect('data/ratingRager.db')
cursor = conn.cursor()

for index, row in df.iterrows():
    cursor.execute(
        "UPDATE scores SET overall_score = ? WHERE restaurant_id = ?;",
        (row['overall_score'], row['restaurant_id'])
    )

conn.commit()
conn.close()

In [ ]:
QUERY = """
    SELECT r.id, r.name, s.overall_score FROM restaurants as r
    JOIN scores as s ON r.id=s.restaurant_id
    ORDER BY s.overall_score DESC
"""
df = print_table(QUERY)


In [ ]:
conn = sqlite3.connect('data/ratingRager.db')
cursor = conn.cursor()
for index, row in df.iterrows():
    cursor.execute("UPDATE restaurants SET ranking = ? WHERE id = ?", (index+1, row['id']))

conn.commit()
conn.close()

In [ ]:
QUERY = """
    SELECT r.id, r.name, s.overall_score, r.ranking FROM restaurants as r
    JOIN scores as s ON r.id=s.restaurant_id
    WHERE overall_score > 0.8
    ORDER BY s.overall_score DESC;
"""
df = print_table(QUERY)


In [ ]:
import folium

m = folium.Map(location=[51.0447, -114.0719], zoom_start=12)  # Coords for Calgary

# Add restaurants as points
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=5,
        color='blue',
        fill=True,
        fill_opacity=0.6,
        popup=folium.Popup(f"Name: {row['name']}<br>Ranking: {row['ranking']} <br>Overall Score: {row['overall_score']:.2f}", max_width=200),
    ).add_to(m)

# Display the map
m.save('map.html')
m
